# GradCAM Visualization Demo with VGG16

Requirement:

* GPU Memory: 6GB or higher

In [1]:
# Replace vanila relu to guided relu to get guided backpropagation.
import tensorflow as tf

from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops

@ops.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    return tf.where(0. < grad, gen_nn_ops._relu_grad(grad, op.outputs[0]), tf.zeros(grad.get_shape()))

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from model import vgg16
import utils

# Create mini-batch for demo

img1 = utils.load_image("./demo.png")
img2 = utils.load_image("./shihtzu_mypuppy.jpg")
img3 = utils.load_image("./tiger.jpg")

batch1_img = img1.reshape((1, 224, 224, 3))
batch1_label = np.array([1 if i == 242 else 0 for i in range(1000)])  # 1-hot result for Boxer
batch1_label = batch1_label.reshape(1, -1)

batch2_img = img2.reshape((1, 224, 224, 3))
batch2_label = np.array([1 if i == 155 else 0 for i in range(1000)])  # 1-hot result for Shih-Tzu
batch2_label = batch2_label.reshape(1, -1)

batch3_img = img3.reshape((1, 224, 224, 3))
batch3_label = np.array([1 if i == 292 else 0 for i in range(1000)])  # 1-hot result for tiger
batch3_label = batch3_label.reshape(1, -1)

batch_img = np.concatenate((batch1_img, batch2_img, batch3_img), 0)
batch_label = np.concatenate((batch1_label, batch2_label, batch3_label), 0)

batch_size = 3

for i in range(batch_size):
    print('See visualization of below category')
    utils.print_prob(batch_label[i], './synset.txt')

# Create tensorflow graph for evaluation
eval_graph = tf.Graph()
with eval_graph.as_default():
    with eval_graph.gradient_override_map({'Relu': 'GuidedRelu'}):
    
        images = tf.placeholder("float", [batch_size, 224, 224, 3])
        labels = tf.placeholder(tf.float32, [batch_size, 1000])
        train_mode = tf.placeholder(tf.bool)

        vgg = vgg16.Vgg16()
        
        vgg.build(images, train_mode)
        cost = tf.reduce_sum((vgg.prob - labels) ** 2)
        train = tf.train.GradientDescentOptimizer(0.0001).minimize(cost)

        # Get last convolutional layer gradient for generating gradCAM visualization
        target_conv_layer = vgg.pool5
        target_conv_layer_grad = tf.gradients(cost, target_conv_layer)[0]

        # Guided backpropagtion back to input layer
        gb_grad = tf.gradients(cost, images)[0]

        # Normalizing the gradients    
        target_conv_layer_grad_norm = tf.div(target_conv_layer_grad, tf.sqrt(tf.reduce_mean(tf.square(target_conv_layer_grad))) + tf.constant(1e-5))


        init = tf.global_variables_initializer()

        
# Run tensorflow 

with tf.Session(graph=eval_graph) as sess:    
    sess.run(init)
    
    prob = sess.run(vgg.prob, feed_dict={images: batch_img, train_mode: False})
    
    gb_grad_value, target_conv_layer_value, target_conv_layer_grad_value = sess.run([gb_grad, target_conv_layer, target_conv_layer_grad_norm], feed_dict={images: batch_img, labels: batch_label, train_mode: True})
    
    for i in range(batch_size):
        utils.print_prob(prob[i], './synset.txt')
        utils.visualize(batch_img[i], target_conv_layer_value[i], target_conv_layer_grad_value[i], gb_grad_value[i])
    

/home/naver/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


See visualization of below category
Top1:  n02108089 boxer 1
Top5:  [('n02108089 boxer', 1), ('n15075141 toilet tissue, toilet paper, bathroom tissue', 0), ('n02319095 sea urchin', 0), ('n02395406 hog, pig, grunter, squealer, Sus scrofa', 0), ('n02391049 zebra', 0)]
See visualization of below category
Top1:  n02086240 Shih-Tzu 1
Top5:  [('n02086240 Shih-Tzu', 1), ('n02321529 sea cucumber, holothurian', 0), ('n02396427 wild boar, boar, Sus scrofa', 0), ('n02395406 hog, pig, grunter, squealer, Sus scrofa', 0), ('n02391049 zebra', 0)]
See visualization of below category
Top1:  n02129604 tiger, Panthera tigris 1
Top5:  [('n02129604 tiger, Panthera tigris', 1), ('n15075141 toilet tissue, toilet paper, bathroom tissue', 0), ('n02319095 sea urchin', 0), ('n02391049 zebra', 0), ('n02389026 sorrel', 0)]
/home/naver/Documents/gradcam_opensource/model/vgg16.npy
npy file loaded
build model started
build model finished: 0s


ResourceExhaustedError: OOM when allocating tensor of shape [3,3,512,512] and type float
	 [[Node: conv4_3/filter/initial_value = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [3,3,512,512] values: [[[-0.0041355654 0.007471954 -0.0035011589]]]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'conv4_3/filter/initial_value', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/naver/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/naver/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/naver/.local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/naver/.local/lib/python3.5/site-p